In [2]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [12]:
from langchain_core.tools import tool
from langchain_chroma import Chroma


from dotenv import load_dotenv
import sys
import os
sys.path.append('../')
from embedding.call_embedding import get_embedding
load_dotenv()

embedding = get_embedding("m3e")
persist_directory = '../vector_db/chroma'

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    print(f"Retrieving information for query: {query}")
    vectordb = Chroma(persist_directory=persist_directory,embedding_function=embedding)
    retrieved_docs = vectordb.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [4]:
from langchain_community.chat_models import ChatZhipuAI
llm=ChatZhipuAI(
    model="glm-4-long",
    temperature=0.1,
)

In [6]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    print(response)
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [7]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [8]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hello
content='Hello! How can I assist you today?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 139, 'total_tokens': 150}, 'model_name': 'glm-4-long', 'finish_reason': 'stop'} id='run-cd2a5139-ec85-49f2-822f-080b396042c6-0'
================================== Ai Message ==================================

Hello! How can I assist you today?


In [13]:
input_message = "什么是蘑菇书(easyrl)？"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

什么是蘑菇书(easyrl)？
content='' additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "什么是蘑菇书(easyrl)？"}', 'name': 'retrieve'}, 'id': 'call_-8790938269921842468', 'index': 0, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 146, 'total_tokens': 162}, 'model_name': 'glm-4-long', 'finish_reason': 'tool_calls'} id='run-08e21664-ea8b-424c-a237-e2495298a6a4-0' tool_calls=[{'name': 'retrieve', 'args': {'query': '什么是蘑菇书(easyrl)？'}, 'id': 'call_-8790938269921842468', 'type': 'tool_call'}]
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_-8790938269921842468)
 Call ID: call_-8790938269921842468
  Args:
    query: 什么是蘑菇书(easyrl)？
================================= Tool Message =================================
Name: retrieve

Source: {'category': 'Title', 'element_id': 'c77db7741393ef5b11ce8739

In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [14]:
input_message = "llm_universe是什么？"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

llm_universe是什么？
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_-8790921571086572865)
 Call ID: call_-8790921571086572865
  Args:
    query: llm_universe是什么
================================= Tool Message =================================
Name: retrieve

Source: {'source': './knowledge_db/readme_summary/llm-universe_summary.md'}
Content: llm-universe Summary

这个仓库名是 llm-universe. 这个仓库主要是关于动手学习大模型应用开发的教程，介绍了大模型的简介、API调用、开发流程、数据库搭建、Prompt设计、验证迭代、前后端开发等内容，旨在帮助小白开发者快速掌握大模型应用开发的基础技能。

Source: {'source': './knowledge_db/readme_summary/llms-from-scratch-cn_summary.md'}
Content: llms-from-scratch-cn Summary

这个仓库名是 llms-from-scratch-cn. 这仓库内容主要是一个详细教程，教你如何从头开始实现类似ChatGPT的大语言模型（LLM）。内容涵盖了编码、预训练和微调过程，适合有编程基础的人深入了解LLM工作原理，并愿意从零开始构建和训练自己的LLM。总体而言，这是一个系统化学习路径，强调实践和深入理解模型原理的项目。
================================== Ai Message ==================================


In [16]:
input_message = "可以详细介绍一下吗"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

可以详细介绍一下吗
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_-8790933425198111245)
 Call ID: call_-8790933425198111245
  Args:
    query: llm-universe
================================= Tool Message =================================
Name: retrieve

Source: {'source': './knowledge_db/readme_summary/llm-universe_summary.md'}
Content: llm-universe Summary

这个仓库名是 llm-universe. 这个仓库主要是关于动手学习大模型应用开发的教程，介绍了大模型的简介、API调用、开发流程、数据库搭建、Prompt设计、验证迭代、前后端开发等内容，旨在帮助小白开发者快速掌握大模型应用开发的基础技能。

Source: {'source': './knowledge_db/readme_summary/llms-from-scratch-cn_summary.md'}
Content: llms-from-scratch-cn Summary

这个仓库名是 llms-from-scratch-cn. 这仓库内容主要是一个详细教程，教你如何从头开始实现类似ChatGPT的大语言模型（LLM）。内容涵盖了编码、预训练和微调过程，适合有编程基础的人深入了解LLM工作原理，并愿意从零开始构建和训练自己的LLM。总体而言，这是一个系统化学习路径，强调实践和深入理解模型原理的项目。
================================== Ai Message ==================================

llm-unive